In [1]:
# VYTVORENIE CSV SUBOROV S PREDIKOVANYMI A SKUTOCNYMI HODNOTAMI PARAMETROV 
# PRE SPATNE VYKRESLENIE KRIVIEK SYSTEMOM ELISA

In [2]:
# BLOK 1
# Nacitanie kniznic

import numpy as np
import pandas as pd
import pickle
import math
from ast import literal_eval

2023-05-09 15:05:58.456635: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# funkcia pre vytvorenie dat na plotovanie

def make_prediction_observed(df, name_of_df, regr_method_folder, model):
    df = df.copy()
    # vytvorenie pola kriviek
    X_o = []
    for row in df["Curve"]:
        processed_data=literal_eval(row)
        X_o.append(processed_data)
    X_o=np.array(X_o, dtype=np.float32)

    # predikcia
    y_pred = model.predict(X_o)
    
    # vytvorenie df
    denorm_df = pd.DataFrame(y_pred, columns = [
                        "pred_t2/t1",
                        "pred_inc",
                        "pred_q",
                        "pred_omega1",
                        "pred_omega2"
                        ])
    df["pred_inc"] = denorm_df["pred_inc"].values
    df["pred_q"] = denorm_df["pred_q"].values
    df["pred_omega1"] = denorm_df["pred_omega1"].values
    df["pred_omega2"] = denorm_df["pred_omega2"].values
    df["pred_t2_t1"] = denorm_df["pred_t2/t1"].values
    df.to_csv(f'data_to_plot/{regr_method_folder}/{name_of_df}.csv')

In [4]:
# BLOK 2
# Nacitanie a spracovanie observacnych dat

obs=pd.read_csv("observed.csv")
obs["T2/T1"]=obs["T2"]/obs["T1"]
obs["Inc"]=(obs["Inc"]*math.pi)/180.0

obs=obs[["T2/T1", "Inc", "Q", "Omega1", "Omega2", "Curve", "Type", "Name", "Filter", "Spot"]]
obs_det = obs.loc[obs.Type.isin(["detached", "detected"])]
obs_ove = obs.loc[obs.Type.isin(["overcontact"])]
obs = obs.reset_index()

In [5]:
# BLOK 3
# Rozdelenie obs. dat do skupin podla filtra

data_to_predict_det = obs_det[["Name", "Curve", "Filter", "Inc", "Q", "Omega1", "Omega2", "T2/T1"]]
data_to_predict_ove = obs_ove[["Name", "Curve", "Filter", "Inc", "Q", "Omega1", "Omega2", "T2/T1"]]

det_data_Bessell_V = data_to_predict_det[data_to_predict_det["Filter"] == "V"]
det_data_Bessell_I = data_to_predict_det[data_to_predict_det["Filter"] == "I"]
det_data_Bessell_R = data_to_predict_det[data_to_predict_det["Filter"] == "R"]
det_data_Bessell_B = data_to_predict_det[data_to_predict_det["Filter"] == "B"]

ove_data_Bessell_V = data_to_predict_ove[data_to_predict_ove["Filter"] == "V"]
ove_data_Bessell_I = data_to_predict_ove[data_to_predict_ove["Filter"] == "I"]
ove_data_Bessell_R = data_to_predict_ove[data_to_predict_ove["Filter"] == "R"]
ove_data_Bessell_B = data_to_predict_ove[data_to_predict_ove["Filter"] == "B"]

In [6]:
# BLOK 4
# nacitanie a predikcia parametrov observacnych systemov metodou Knn

In [5]:
# nacitanie Knn modelov

model_obs_det = pickle.load(open("models/knn_detached_noise_model.pkl", 'rb'))
model_obs_ove = pickle.load(open("models/knn_overcontact_noise_model.pkl", 'rb'))

In [7]:
# Predikcia parametrov observacnych systemov metodou Knn pre kazdy filter zvlast. 
# Vytvorenie suboru .csv so skutocnymi a predikovanymi hodnotami pre kazdy filter zvlast

make_prediction_observed(det_data_Bessell_V, "obs_det_data_Bessell_V_knn", "knn", model_obs_det)
make_prediction_observed(det_data_Bessell_R, "obs_det_data_Bessell_R_knn", "knn", model_obs_det)
make_prediction_observed(det_data_Bessell_I, "obs_det_data_Bessell_I_knn", "knn", model_obs_det)
make_prediction_observed(det_data_Bessell_B, "obs_det_data_Bessell_B_knn", "knn", model_obs_det)

make_prediction_observed(ove_data_Bessell_V, "obs_ove_data_Bessell_V_knn", "knn", model_obs_ove)
make_prediction_observed(ove_data_Bessell_R, "obs_ove_data_Bessell_R_knn", "knn", model_obs_ove)
make_prediction_observed(ove_data_Bessell_I, "obs_ove_data_Bessell_I_knn", "knn", model_obs_ove)
make_prediction_observed(ove_data_Bessell_B, "obs_ove_data_Bessell_B_knn", "knn", model_obs_ove)

In [7]:
# BLOK 5
# nacitanie a predikcia parametrov observacnych systemov metodou SVR

In [45]:
# nacitanie Svr modelov

model_obs_det = pickle.load(open("models/svr_detached_model.pkl", 'rb'))
model_obs_ove = pickle.load(open("models/svr_overcontact_model.pkl", 'rb'))

In [49]:
# Predikcia parametrov observacnych systemov metodou SVR pre kazdy filter zvlast. 
# Vytvorenie suboru .csv so skutocnymi a predikovanymi hodnotami pre kazdy filter zvlast

make_prediction_observed(det_data_Bessell_V, "obs_det_data_Bessell_V_svr", "svr", model_obs_det)
make_prediction_observed(det_data_Bessell_R, "obs_det_data_Bessell_R_svr", "svr", model_obs_det)
make_prediction_observed(det_data_Bessell_I, "obs_det_data_Bessell_I_svr", "svr" ,model_obs_det)
make_prediction_observed(det_data_Bessell_B, "obs_det_data_Bessell_B_svr", "svr", model_obs_det)

make_prediction_observed(ove_data_Bessell_V, "obs_ove_data_Bessell_V_svr", "svr", model_obs_ove)
make_prediction_observed(ove_data_Bessell_R, "obs_ove_data_Bessell_R_svr", "svr", model_obs_ove)
make_prediction_observed(ove_data_Bessell_I, "obs_ove_data_Bessell_I_svr", "svr", model_obs_ove)
make_prediction_observed(ove_data_Bessell_B, "obs_ove_data_Bessell_B_svr", "svr", model_obs_ove)